platformio run -target upload


esp32 connects to via bluetooth to your phone
esp32 creates access point with its name & password the same. starting w 'ESP32'
Figure out if thats sufficient... 
Anyone who can connect to your wifi may subscribe for the AP's publishings
... Only the Access Points struggle to communicate with eachother...
.. They can communicate with eachther using the mesh network ...
... but take orders from bluetooth or mqtt.
esp32 Mesh

first node will look for other nodes on the mesh
if no mesh exists meeting the nodes specific requirements
-- The node will Broadcast BT 
-- The node will Create its own Mesh using Defaults
removing a node from the mesh will respond -> indicating to the user


Every node in ESP-MESH that is able to form downstream connections (i.e. has a softAP interface) will periodically transmit Wi-Fi beacon frames
A node uses beacon frames to allow other nodes to detect its presence and know of its status.
 Idle nodes will listen for beacon frames to generate a list of potential parent nodes, one of which the idle node will form an upstream connection with.

 The routing table of a particular node will consist of the MAC addresses of all nodes within the particular node’s subnetwork (including the MAC address of the particular node itself). Each routing table is internally partitioned into multiple subtables with each subtable corresponding to the subnetwork of each child node.


How do users pair with eachother?
Nodes scan for other APs 
-- inform the root node who informs the owner
-- nodes can multi 'mesh' and merge information across networks


Satellite or Command

Raspberry Pi to flash binaries
https://www.esp32.com/viewtopic.php?t=10227

a flooding approach based on MAC to identify nodes will be used for configuring and assigning IP addresses and range.
Gateway is the responsible node for setting up the network. Mesh network is administered purely by MAC-based flooding. IP is only used for routing the regular packets, IP address 192.168.0.0 is for the gateway and 192.168.255.255 is for broadcast.
I want the ESP32 to act as a router with a DHCP
What I want help with is -
1) How to connect an ESP32 to multiple ESP32's (for MAC based flooding)
2) How to receive a packet and simply route them without any delays
compatible with TCP/IP and must route using IP addresses
MAC routing and as the number of nodes increases the space-efficiency is inversely affected. While the routing scheme designed by me is specifically designed to be more efficient and space conserving. How bigger the network grows the space complexity will remain a constant in my network [which I intend to build ](however the maximum number of nodes is 250). Since the routing is based on TCP/IP header, it is also interoperable with the existing TCP/IP networks.

Plan

```
getName()
setName()
getWifiNamePass
setWifiNamePass
getDeviceName
setDeviceName

SubscribeTo
BroadcastTo
usersName

MidiEvent() -> *keypress djpad* -> AudioEvent() 
AudioEvent -> Meyda() 
Meyda -> LightEvent() 

LightEvent -> audioFX(userName, [colors, duration, variation, intensity, transition])



Slider Knobs -> Volume Hi Mid Low
Stop/ Start W/ Display

Consider: OTA, Flash Defaults SPIFFS, Broad/Multi/Uni, Chip/Module/Board

Plan Part 2

- ColorWheel Maps to Octaves. Avg top 3 freq/colors

1. Only Admin may broadcast
2. gatway sends to children
3. children never message siblings
4. only msg 1 relative per bc & prefer to send to gateway
5. children deliver inbound messages to gateway
6. gateway delivers data to admin... go to step 1...
- Broadcasts are limited by # hops. #hops is managed by GW?



```
On start look to storage for settings.
try: connect to preferred network
Except: 
  scanorEstablishDefaultNetwork()
DefaultNetworkLoop():
  if isDisconnected(): ScanorestablishDefaultNetwork()
```

userA -> gatewayA
userB -> gatewayB
userC -> gatewayC

gatewayA -> nodeA1
gatewayB -> nodeB1
gatewayC -> nodeC1

gatewayA -> nodeA2
gatewayB -> nodeB2
gatewayC -> nodeC2

gatewayA -> nodeA3
gatewayB -> nodeB3
gatewayC -> nodeC3

gatewayA -> gatewayB
gatewayB -> gatewayA
gatewayB -> gatewayC
gatewayC -> gatewayB

nodeA3 -> nodeB1
nodeB1 -> nodeA3

nodeC3 -> nodeB3
nodeB3 -> nodeC3





- gateway broadcasts user cmds. leaf's unicast out farther but not to eachother.
- gateway and leafs unicast recieved msgs to user if not from the user
- Do not relay msgs from anyone other than the user
- user may choose to relay a msg it recieved from someone else.

pub/sub should allow you to act simply as a router or to extend the pub/sub by replicating and posting the message as well.

Command and satellite must share same channel.

Multi-Acess Frequency Hopping allows for a shared transmission medium

1. 1Peer FF:FF:FF:FF -> Broadcast to all peers on ch -> esp32

2. Set Peer Mac Addr -> Broadcast by sending to common mac addr -> ESP32

3. Adhoc Addressing Layer

Frequency hoping, channels limit # conn, multi access method requires
- macaddr of other devices
- or soft ap sharing
- or bt sharing
- or sendbroadcast to all on network

How it works

- Command->STA->WifiMacAddr()
- Satellite->AP->WifiSoftAPMacAddr()
- 0-14ch: 2.4GHZ: 802.11.n 
- Max paired devices = 20. Sent from STA or AP modes. 
- - -> circumvent unicasting by setting all nodes to have the same MAC ADDR 

- Service/ Character
- EspOwnerName
- EspDeviceName
- Pin 13 2
- LED #
- SSIDConnectorNamePrefix

ifUpdates->Broadcast
onReception => 
1. update State of leafnode values
2. get rout node values
3. subscriptionValues
 - Send data 2 clients so they may "subscribe".
 - Leaf nodes need a log(val @ timestamp)





## Setup
1. ConfigDeviceAP (mac=ssid, pass=1234, print-yay!)
2. RegisterCallbacks (msgsnt,msgrcvd-write2pin)
3. Init ESPNow(onMsgSnt, OnMsgRcvd): refers to step 2
 - strip.begin
 - strip.show
 - BLEESP = ESPName
 - BLEESP.CreateService .createCharacteristic
 - BLEESP Register Callback
 - BLEESP Set Value("HelloWorld")
 - BLEESP.start

## Loop
41. Scan4LeafNodes
 - Wifi.ScanNetwork
 - For Node in AP = add to list
42. Manage Leaf Nodes
 - For each leaf node: if name = "espname": check if paired. 
43. Send Data - ESP
44. If Command = XYZ


1. OnBLERecieved: if leafnodes -> ESPNow Broadcast. Run Command
2. ONESPRecieved: if BLE!connected:DoNothing. Else: if send="espname:donothing" else:"send2ble"

#### Config Device AP
- scan4leafnodes 
 - if a node with esp.bt exists and you are not connected to the user then there is already a gateway and bluetooth is not needed for thihs node.
- manage leaf nodes 
 - try to connect to user usin bt. if success then change wifi name to include bt so that other nodes know that you are the gateway and they do not need to connect.
- espnow_senddata(command) 
 - if not leader do not send
 - only if leader. or send the data but only read from leaders & leaders would read from everyone...

if btmessage -> relay out
if publicmsg -> if msg is leader: run cmd & broadcast. else: if in group do nothing



#### Partition Table
Includes root. values from different people + all current data to be fwded from leafs to rest

Group: Updated: Val

Leafid: Updated: Val

#### How MSG Works

Within body of a node with an ssid=espname... Body(rootnodessid, rootsentthis@, command)
- msg recv & isroot?
- 1. & root nodessid = espname+bt
- - runbroadcast, update timestamp, is command the same as before? 
- - check timestamp, same broadcast? relay nothing
- 2. or root nodessid = espname only
- - patitiontable: groupname:command:timestamp
- 3. or root nodessid != espname... do nothing.